In [9]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_excel(r"C:\Users\eugin\OneDrive\Документы\Traning\POWER_BI\MF_Training\PROJECTS\Mock_Sales\Project_1_2023_Mock_Set\Sales_Report_Slim.xlsx")

In [21]:
df.head()

,Date,Revenue
0,2023-01-01,7690.51
1,2023-01-01,2386.64
2,2023-01-01,3680.78
3,2023-01-01,6213.88
4,2023-01-01,8205.46


In [23]:
# Ensure datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])

In [25]:
# Confidence interval prep
df['Month'] = df['Date'].dt.to_period('M')
monthly = df.groupby('Month')['Revenue'].sum().reset_index()
monthly['Month'] = monthly['Month'].dt.to_timestamp()

In [27]:
monthly['RollingMean'] = monthly['Revenue'].rolling(window=3).mean()
monthly['RollingStd'] = monthly['Revenue'].rolling(window=3).std()

In [29]:
monthly['CI_Upper'] = monthly['RollingMean'] + 1.96 * monthly['RollingStd']
monthly['CI_Lower'] = monthly['RollingMean'] - 1.96 * monthly['RollingStd']

In [31]:
# Optional labeling for Matrix
monthly['CI_Flag'] = monthly.apply(
    lambda row: 'Above CI' if row['Revenue'] > row['CI_Upper']
    else 'Below CI' if row['Revenue'] < row['CI_Lower']
    else 'Within CI',
    axis=1
)

In [35]:
monthly.head()

,Month,Revenue,RollingMean,RollingStd,CI_Upper,CI_Lower,CI_Flag
0,2023-01-01,864262.97,NaN,NaN,NaN,NaN,Within CI
1,2023-02-01,672499.73,NaN,NaN,NaN,NaN,Within CI
2,2023-03-01,657819.71,731527.470000,115186.416157,957292.845667,505762.094333,Within CI
3,2023-04-01,726656.34,685658.593333,36255.857052,756720.073156,614597.113511,Within CI
4,2023-05-01,754676.69,713050.913333,49841.241656,810739.746978,615362.079688,Within CI


In [44]:
# Saving file
from pathlib import Path

target_dir = Path(r"C:\Users\eugin\OneDrive\Документы\Traning\POWER_BI\MF_Training\PROJECTS\Mock_Sales\Project_1_2023_Mock_Set")
file_path = target_dir / "CI_Analysis.csv"
monthly.to_csv(file_path, index=False)